In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("dovlt_budcsi._xrclr__xususi_ckisi__20250410120531.csv")
df.head()

,Mövzu,2020,2021,2022,2023,2024
0,Xərclər-cəmi (faiz),100.0,100.0,100.0,100.0,100.0
1,Ümumi dövlət xidmətləri (faiz),13.4,14.5,12.8,10.3,10.9
2,Müdafiə və milli təhlükəsizlik (faiz),14.2,16.6,15.3,16.1,17.4
3,"Məhkəmə hakimiyyəti, hüquq-mühafizə və prokuro...",6.9,7.4,7.4,7.3,7.5
4,Təhsil (faiz),10.5,11.3,11.5,11.3,11.6


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mövzu   13 non-null     object 
 1   2020    13 non-null     float64
 2   2021    13 non-null     float64
 3   2022    13 non-null     float64
 4   2023    13 non-null     float64
 5   2024    13 non-null     float64
dtypes: float64(5), object(1)
memory usage: 756.0+ bytes
